<a href="https://colab.research.google.com/github/surim1103/solidity/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%95%94%ED%98%B8%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pycryptodomex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
!pip uninstall crypto
!pip uninstall pycrypto
!pip install pycrypto

Found existing installation: pycrypto 2.6.1
Uninstalling pycrypto-2.6.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/Crypto/*
    /usr/local/lib/python3.7/dist-packages/pycrypto-2.6.1.dist-info/*
Proceed (y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/uninstall.py", line 86, in run
    auto_confirm=options.yes, verbose=self.verbosity > 0,
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_install.py", line 658, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_uninstall.py", line 380, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_uninstall.py", l

In [22]:
import base64
from Crypto import Random
from Crypto.Cipher import AES

블럭사이즈에 대한 패딩로직-영문과 기호는 문자당 1바이트지만 한글은 문자당 2바이트이기 때문이다. len()함수를 활용해 길이를 통해 바이트 계산을 하는 방식이므로 ‘utf-8’ 변환하지 않을 경우 오류가 발생하게 된다.

In [23]:
BS = 16
pad = lambda s: s + (BS - len(s.encode('utf-8')) % BS) * chr(BS - len(s.encode('utf-8')) % BS)
unpad = lambda s : s[:-ord(s[len(s)-1:])]

암호화 알고리즘 클래스 생성

In [24]:
class AESCipher:
    def __init__( self, key ):
        self.key = key

    def encrypt( self, raw ):
        raw = pad(raw)
        iv = Random.new().read( AES.block_size )
        cipher = AES.new( self.key, AES.MODE_CBC, iv )
        return base64.b64encode( iv + cipher.encrypt( raw.encode('utf-8') ) )

    def decrypt( self, enc ):
        enc = base64.b64decode(enc)
        iv = enc[:16]
        cipher = AES.new(self.key, AES.MODE_CBC, iv )
        return unpad(cipher.decrypt( enc[16:] ))

KEY설정- AES256을 구현하기 위해 32바이트의 키를 생성한다.

In [26]:
key = [0x10, 0x01, 0x15, 0x1B, 0xA1, 0x11, 0x57, 0x72, 0x6C, 0x21, 0x56, 0x57, 0x62, 0x16, 0x05, 0x3D,
        0xFF, 0xFE, 0x11, 0x1B, 0x21, 0x31, 0x57, 0x72, 0x6B, 0x21, 0xA6, 0xA7, 0x6E, 0xE6, 0xE5, 0x3F]

데이터 암호화

In [27]:
data = "202021577 함수림"

In [28]:
encrypted_data = AESCipher(bytes(key)).encrypt(data)  
encrypted_data

b'4H1RzCe9YfzvVpdBaCm3THsFKXLQAc4wUoVbGxEz8PoSChSDuvfT01RBsVy7AtR4'

데이터 복호화

In [29]:
decrypted_data = AESCipher(bytes(key)).decrypt(encrypted_data)
decrypted_data.decode('utf-8')

'202021577 함수림'